In [1]:
# start time - 1609459200000, 1641028611000, 1672564611000, 1654045261000, 1651366861000 (2021, 2022, 2023)

In [2]:
# ! conda install -c conda-forge ta --yes

In [3]:
import requests
import pandas as pd
import ta
import datetime as dt
import numpy as np
import time
import itertools
import os

In [4]:
from binance.client import Client
from dotenv import load_dotenv

# Get the path to the current directory
current_directory = os.getcwd()

# Specify the path to the .env file relative to the current directory
dotenv_path = os.path.join(current_directory, '.env')

# Load the environment variables from the .env file
load_dotenv(dotenv_path)

api_key = os.getenv('API_KEY')
api_secret = os.getenv('SECRET_KEY')

client = Client(api_key, api_secret, testnet = False)

In [5]:
interval_arr = ['15m', '1h']
ema_arr = [8, 18, 38]

ep_per_arr = [1.005, 0.999, 0.995, 0.990]

sl_per_arr = [0.999, 1.003, 1.007, 1.011]

# stop loss determine candlestick
sl_det_arr = ['Close', 'High']

In [6]:
timezone = 8
# symbol = 'ethusdt'
symbol_arr = ['ethusdt', 'crvusdt', 'injusdt', 'imxusdt', 'roseusdt', 'linausdt', 'ftmusdt', 'adausdt', 'avaxusdt']

start_time = 1635743450000
end_time = round(time.time() * 1000)

def set_time():
    global start_time, end_time
    start_time = 1635743450000
    end_time = round(time.time() * 1000)

# step between timestamps in milliseconds
step = 1000

In [7]:
dataframes = {}
def create_raw(symbol_arr, interval_arr, step):
    global start_time, end_time
    
    for interval in interval_arr:
        for symbol in symbol_arr:
            set_time()
            # Fetch the data using batch requests
            data = []
            while start_time < end_time:
                limit = min(step, end_time - start_time + 1)  # Adjust the limit for the last batch
                response = client.get_klines(symbol=symbol.upper(), interval=interval, limit=limit, startTime=start_time)

                if len(response) == 0:
                    break  # No more data available, exit the loop
                data.extend(response)
                start_time = response[-1][0] + 1

            # Convert the data to a DataFrame
            columns = [
                "Open_Time", "Open", "High", "Low", "Close", "Volume", "Close_Time",
                "Quote asset volume", "Number of trades", "Taker buy base asset volume",
                "Taker buy quote asset volume", "Ignore"
            ]
            raw_df = pd.DataFrame(data, columns=columns)   

            raw_df = raw_df[['Open_Time', 'Open', 'Close', "High", "Low", 'Volume']]

            dataframes[f'df_{interval}_{symbol}'] = raw_df

In [8]:
loop_start_time = time.time()
create_raw(symbol_arr, interval_arr, step)
loop_end_time = time.time()
print("Time taken to execute for loop:", loop_end_time - loop_start_time, "seconds")
print(dataframes)

Time taken to execute for loop: 57.115670919418335 seconds
{'df_15m_ethusdt':            Open_Time           Open          Close           High  \
0      1635743700000  4233.64000000  4238.15000000  4246.31000000   
1      1635744600000  4238.16000000  4251.22000000  4251.54000000   
2      1635745500000  4251.23000000  4225.00000000  4252.58000000   
3      1635746400000  4225.00000000  4209.00000000  4225.99000000   
4      1635747300000  4208.99000000  4232.36000000  4234.21000000   
...              ...            ...            ...            ...   
56157  1686289500000  1837.67000000  1839.45000000  1839.73000000   
56158  1686290400000  1839.45000000  1837.60000000  1839.45000000   
56159  1686291300000  1837.59000000  1840.61000000  1842.00000000   
56160  1686292200000  1840.61000000  1840.67000000  1840.67000000   
56161  1686293100000  1840.67000000  1839.50000000  1840.69000000   

                 Low         Volume  
0      4231.60000000  2158.18360000  
1      4238.14000

In [9]:
def get_klines(symbol, interval):
    global dataframes
    df = dataframes[f'df_{interval}_{symbol}']
    df = df[['Open_Time', 'Open', 'Close', "High", "Low", 'Volume']].astype(float)
    df = df.set_index('Open_Time')

    df.index = pd.to_datetime(df.index, unit='ms') + pd.Timedelta(hours=timezone)
    df = df[~df.index.duplicated(keep='first')]

    return df

In [12]:
symbol_dfs = {}
def multi_timeframes(symbol):
    df = get_klines(symbol, interval_arr[0]).copy() # eth 1h
    df = df.astype(float).round(4)

    for interval in interval_arr:
        raw_df = get_klines(symbol, interval)
        # ema
        for ema in ema_arr:
            column_name = f'ema_{ema}_{interval}' # eth 1h 4th ema
            df[column_name] = ta.trend.EMAIndicator(raw_df.Close, window=ema, fillna=True).ema_indicator()

        # rsi
    #     rsi = ta.momentum.RSIIndicator(raw_df.Close, window = rsi_int)
    #     df[f'rsi_{interval}'] = rsi.rsi()

        # atr
    #         df['atr'] = ta.volatility.average_true_range(df.High, df.Low, df.Close)


    # reset index and set current index as a column
    df = df.reset_index()

    # set new index with integers
    df = df.set_index(pd.RangeIndex(len(df)))

    # time_format(timezone)
    df['Open_Time'] = df['Open_Time'].dt.strftime('%Y-%m-%d %H:%M:%S')

    # fill up higher time frame empty values with equal interval between each value
    df = df.replace('', np.nan)
    df['ema_8_1h'] = df['ema_8_1h'].interpolate()
    df['ema_18_1h'] = df['ema_18_1h'].interpolate()
    df['ema_38_1h'] = df['ema_38_1h'].interpolate()
    symbol_dfs[f'df_{symbol}'] = df
    return symbol_dfs[f'df_{symbol}']

In [13]:
multi_timeframes(symbol_arr[1])
print(symbol_dfs)

{'df_crvusdt':                  Open_Time   Open  Close   High    Low   Volume  ema_8_15m  \
0      2021-11-01 13:15:00  4.324  4.337  4.353  4.313  90406.2   4.337000   
1      2021-11-01 13:30:00  4.336  4.347  4.348  4.317  52232.3   4.339222   
2      2021-11-01 13:45:00  4.348  4.301  4.348  4.296  95230.5   4.330728   
3      2021-11-01 14:00:00  4.301  4.281  4.311  4.276  75623.1   4.319678   
4      2021-11-01 14:15:00  4.282  4.312  4.317  4.270  55221.5   4.317971   
...                    ...    ...    ...    ...    ...      ...        ...   
56157  2023-06-09 13:45:00  0.758  0.759  0.759  0.758  28983.0   0.758010   
56158  2023-06-09 14:00:00  0.759  0.757  0.759  0.757  22292.5   0.757786   
56159  2023-06-09 14:15:00  0.758  0.759  0.760  0.758  29610.7   0.758055   
56160  2023-06-09 14:30:00  0.760  0.760  0.761  0.759  13919.7   0.758488   
56161  2023-06-09 14:45:00  0.761  0.759  0.761  0.759  14923.4   0.758601   

       ema_18_15m  ema_38_15m  ema_8_1h  ema_18_

In [14]:
# def check_cross(df, kd_dir):
#     up = df['slow_k'] > df['slow_d']
#     down = df['slow_k'] < df['slow_d']
#     if kd_dir == 'Up':
#         return up.diff() & up
#     if kd_dir == 'Any':
#         return up.diff()
#     if kd_dir == 'Down':
#         return down.diff() & down


def indicators(df):

# ema
#     for i in (200, 500, 1000):
#         df['ema_'+str(i)] = ta.trend.ema_indicator(df.Close, window=i)

# atr
#     df['atr'] = ta.volatility.average_true_range(df.High, df.Low, df.Close)
    
# rsi
#     rsi_int = 14
#     df['rsi'] = ta.momentum.RSIIndicator(df['Close'], window = rsi_int).rsi()

# bband
    bb_int = 30
    bb_dev = 2
    bb = ta.volatility.BollingerBands(df['Close'], window=bb_int, window_dev=bb_dev)
    df['bb_u'] = bb.bollinger_hband()
    df['bb_m'] = bb.bollinger_mavg()
    df['bb_l'] = bb.bollinger_lband()

# kd
#     df['slow_k']= ta.momentum.stoch(df['High'], df['Low'], df['Close'], 17, 5)
#     df['slow_d'] = ta.momentum.stoch_signal(df['High'], df['Low'], df['Close'], 17, 5)
    
# kd cross
#     df['kd_cross'] = check_cross(df, kd_dir)

In [15]:
def conditions(df, ep_per):
    df['c1'] = df['ema_8_15m'] <= df['ema_18_15m']
    df['c2'] = df['ema_18_15m'] <= df['ema_38_15m']
    df['c3'] = df['ema_8_1h'] <= df['ema_38_1h']
    
    # signal
    df['signal'] = df['c1'] & df['c2'] & df['c3']
    
    # open_entry
    df['open_entry'] = df['signal'] & (df['High'] >= df['ema_18_1h'] * ep_per)
    
    return df

In [16]:
def entries(df, ep_per, sl_per, sl_det):

    in_position = False
    stop_loss = np.nan
    take_profit = np.nan
    close_val = df['Close']
    open_val = df['Open']
    high_val = df['High']
    ema_18_val = df['ema_18_1h']
    vol_val = df['Volume']

    df['position'] = ''  # Create an empty column for position
    df['take_profit'] = np.nan

    for index, row in df.iterrows():

        if index == 0:
            continue

        elif row['open_entry']:
            df.at[index, 'entry_p'] = ema_18_val.at[index] * ep_per
            df.at[index, 'stop_loss'] = high_val.at[index] * sl_per
            df.at[index, 'take_profit'] = ema_18_val.at[index] * ep_per * 0.9
            df.at[index, 'position'] = 'Short'
            in_position = True
            stop_loss = df.at[index, 'stop_loss']
            take_profit = df.at[index, 'take_profit']

        elif in_position and row[sl_det] >= stop_loss:
            df.at[index, 'position'] = 'Stop'
            in_position = False
            stop_loss = np.nan
            take_profit = np.nan

        # set take profit
        elif in_position and row['Low'] <= row['bb_l']:
            df.at[index, 'position'] = 'Buy'
            in_position = False
            stop_loss = np.nan
            take_profit = np.nan

        elif in_position and (((vol_val.at[index] > vol_val.shift(1).at[index]) &
                              (open_val.shift(1).at[index] > close_val.shift(1).at[index]) &
                              (open_val.at[index] < close_val.at[index]) & 
                              (high_val.at[index] > open_val.shift(1).at[index])) or
                              (high_val.at[index] >= stop_loss)):
            df.at[index, 'position'] = 'Buy'
            in_position = False
            stop_loss = np.nan
            take_profit = np.nan  
            
    df = df[(df['open_entry']) | (df['position'] != '')]

In [17]:
# 部位回測
def backtest(df, symbol, sl_det):

    df = df.reset_index(drop = True)
    df = df[(df['position'] == 'Short') |
                  (df['position'] == 'Buy') |
                  (df['position'] == 'Stop')]

    # 一次進場多少單位
    pos_size = 200

    col = ['Open_Time', 'Open', 'Close', 'High', 'Low', 'position','entry_p', 'stop_loss', 'take_profit']
    pos = df[col]
    pos = pos.reset_index(drop = True)


    for index, row in pos.iterrows():

        current_pos = 0

        # 進場
        if pos.at[index, 'position'] == 'Short':
            pos.at[index, 'size'] = pos_size / pos.at[index, 'entry_p']
            pos.exit_p = np.nan

        # 出場
        if pos.at[index, 'position'] == 'Buy' or pos.at[index, 'position'] == 'Stop':

            # 停利：達成條件時收盤價
            if pos.at[index, 'position'] == 'Buy':
                pos.at[index, 'exit_p'] = pos.at[index, 'Close']
                
#                 for i in range(index -1, -1, -1):
#                     if pos.at[i, 'position'] == 'Buy':
#                         pos.at[index, 'exit_p'] = pos.at[i, 'take_profit']
#                     break

            # 停損：打到進場停損點（往回跌代，直到最近的'Buy'及其'stop_loss'）
            if (pos.at[index, 'position'] == 'Stop') & (sl_det == 'Close'):
                pos.at[index, 'exit_p'] = pos.at[index, 'Close']
                
            if (pos.at[index, 'position'] == 'Stop') & (sl_det == 'High'):
                for i in range(index -1, -1, -1):
                    if pos.at[i, 'position'] == 'Short':
                        pos.at[index, 'exit_p'] = pos.at[i, 'stop_loss']
                    break

            # 計算每次出場部位大小（每次出場皆清倉）
            for i in range(index -1, -1, -1):
                if pos.at[i, 'position'] == 'Short':
                    current_pos += pos.at[i, 'size']
                    if i == 0:
                        pos.at[index, 'size'] = -current_pos
                    else:
                        continue
                else:
                    pos.at[index, 'size'] = -current_pos
                    current_pos = 0
                    break


    # 計算部位價值
    for index, row in pos.iterrows():
        if pos.at[index, 'position'] == 'Short':
            pos.at[index, 'amt'] = round(pos.at[index, 'size'] * pos.at[index, 'entry_p'], 4)
        elif pos.at[index, 'position'] == 'Buy' or pos.at[index, 'position'] == 'Stop':
            pos.at[index, 'amt'] = round(pos.at[index, 'size'] * pos.at[index, 'exit_p'], 4)


    # 若最後一筆為 Buy，移除該單，迭代驗證
    for index, row in pos.iloc[::-1].iterrows():
        if row['position'] == 'Short':
            pos = pos.drop(index)
        else:
            break


    # 手續費、滑點、價差
    fee = 0.05 / 100
    amt_abs_sum = pos.amt.abs().sum()
    ttl_fee = amt_abs_sum * fee

    # 損益
    leverage = 10
    ttl_profit = pos.amt.sum() - ttl_fee

    
    # 計算獲利/虧損次數
    agg_amts = []

    for i in range(len(pos) - 1, -1, -1):

        if pos.loc[i, 'position'] in ['Stop', 'Buy']:

            # look out for the + sign
            total_amt = pos.loc[i, 'amt'] - np.absolute(pos.loc[i, 'amt']) * fee

            # iterate backwards from the current row until reaching another 'Stop' or 'Sell'
            # watch out for the + in total_amt += trading_fee
            j = i - 1
            while j >= 0 and pos.loc[j, 'position'] not in ['Stop', 'Buy']:
                total_amt += pos.loc[j, 'amt']
                trading_fee = np.absolute(pos.loc[j, 'amt']) * fee
                total_amt -= trading_fee
                j -= 1

            # add the aggregated amount to the list
            agg_amts.append(total_amt)

    agg_amts.reverse()

    
    # 計算進場最大部位，最大損益
    consec_entry = 0
    position_amt_sum = 0
    max_consec_entry = 0
    max_position = 0
    max_profit = 0
    max_loss = 0

    for index, row in pos.iterrows():

        if row['position'] == 'Short':

            consec_entry += 1
            position_amt_sum += row['amt']

        elif row['position'] in ['Buy', 'Stop']:

            if consec_entry > max_consec_entry:
                max_consec_entry = consec_entry
                max_position = position_amt_sum

            position_amt_sum += row['amt']

            if position_amt_sum > max_profit:
                max_profit = position_amt_sum

            if position_amt_sum < max_loss:
                max_loss = position_amt_sum

            consec_entry = 0
            position_amt_sum = 0

        else:
            pass


    # 最大部位
    profit_per = "{:.2f}%".format(ttl_profit / (max_position/leverage) * 100)


    # 勝率
    wins = 0
    loses = 0

    for trade in agg_amts:
        if trade > 0:
            wins += 1
        elif trade < 0:
            loses += 1

    win_rate = "{:.2f}%".format(wins / (wins + loses) * 100)

    cumulative_values = []
    cumulative_sum = 0

    for value in agg_amts:
        cumulative_sum += value
        cumulative_values.append(cumulative_sum)

    max_drawdown = min(cumulative_values)
    
    # 結果
    result = {'Profit': [round(ttl_profit, 2)],
              'Fee': [round(ttl_fee, 2)],
              'Max_Profit': [round(max_profit, 2)],
              'Max_Loss': [round(max_loss, 2)],
              'Max_Entry': [max_consec_entry],
              'Max_Position': [round(max_position, 2)],
              'Profit_%': [profit_per],
              'Win_Rate': [win_rate],
              'PF_Ratio': [round(ttl_profit/ttl_fee, 2)],
              'Max_Drawdown': [round(max_drawdown,2)]}


    result_df = pd.DataFrame(result)
    
#     pos.to_csv(f'{symbol}_{sl_det}.csv')

    return result_df, ttl_profit, win_rate, ttl_fee

In [18]:
# iterate using start_time, under same time interval, test different variables
def run_backtest():
    
    results_df = pd.DataFrame()
    
    symbol_list = []
    sl_per_list = []
    ep_per_list = []
    sl_det_list = []

    i = 0

    loop_start_time = time.time()
    
    parameters = [(symbol, sl_per, ep_per, sl_det) 
                  for symbol in symbol_arr
                  for sl_per in sl_per_arr
                  for ep_per in ep_per_arr
                  for sl_det in sl_det_arr]
    
    for symbol, sl_per, ep_per, sl_det in parameters:

        df = multi_timeframes(symbol)
        df = df.reset_index()
        indicators(df)
        conditions(df, ep_per)
        entries(df, ep_per, sl_per, sl_det)
        backtest_df, ttl_profit, win_rate, fee = backtest(df, symbol, sl_det)

        results_df = pd.concat([results_df, backtest_df], ignore_index = True)

        symbol_list.append(symbol)
        sl_per_list.append(sl_per)
        ep_per_list.append(ep_per)
        sl_det_list.append(sl_det)

        i += 1
        
        print(f" {i} - {symbol}, sl: {sl_per}, tp: {ep_per}, {sl_det}, {round(ttl_profit, 2)}, {round(fee, 2)}, {round(ttl_profit/fee, 2)}, {win_rate} ")

    loop_end_time = time.time()

    print("Time taken to execute for loop:", loop_end_time - loop_start_time, "seconds")

    results_df['symbol'] = symbol_list
    results_df['sl_per'] = sl_per_list
    results_df['ep_per'] = ep_per_list
    results_df['sl_det'] = sl_det_list

    results_df = results_df[['symbol',
                             'sl_per', 'ep_per', 'sl_det',
                             'Profit', 'Fee', 'Max_Profit', 'Max_Loss',
                             'Max_Entry', 'Max_Position', 'Profit_%', 'Win_Rate', 'PF_Ratio', 'Max_Drawdown'
                            ]]
    
    return results_df

In [19]:
results_df = run_backtest()
results_df = results_df.sort_values('Profit', ascending = False)
# print(results_df)

 1 - ethusdt, sl: 0.999, tp: 1.005, Close, 190.52, 16.1, 11.84, 68.85% 
 2 - ethusdt, sl: 0.999, tp: 1.005, High, 193.45, 16.1, 12.02, 50.82% 
 3 - ethusdt, sl: 0.999, tp: 0.999, Close, 947.25, 303.17, 3.12, 58.10% 
 4 - ethusdt, sl: 0.999, tp: 0.999, High, 1023.68, 303.14, 3.38, 49.78% 
 5 - ethusdt, sl: 0.999, tp: 0.995, Close, 78.57, 1124.4, 0.07, 50.00% 
 6 - ethusdt, sl: 0.999, tp: 0.995, High, 992.07, 1123.94, 0.88, 46.95% 
 7 - ethusdt, sl: 0.999, tp: 0.99, Close, -4225.14, 1911.56, -2.21, 42.04% 
 8 - ethusdt, sl: 0.999, tp: 0.99, High, -1772.66, 1910.33, -0.93, 41.11% 
 9 - ethusdt, sl: 1.003, tp: 1.005, Close, 188.5, 16.1, 11.71, 63.33% 
 10 - ethusdt, sl: 1.003, tp: 1.005, High, 191.18, 16.1, 11.88, 63.33% 
 11 - ethusdt, sl: 1.003, tp: 0.999, Close, 1005.74, 302.95, 3.32, 63.50% 
 12 - ethusdt, sl: 1.003, tp: 0.999, High, 1039.39, 302.93, 3.43, 63.17% 
 13 - ethusdt, sl: 1.003, tp: 0.995, Close, 283.47, 1124.3, 0.25, 54.93% 
 14 - ethusdt, sl: 1.003, tp: 0.995, High, 580.86

 112 - imxusdt, sl: 1.003, tp: 0.99, High, 1987.92, 1251.58, 1.59, 56.02% 
 113 - imxusdt, sl: 1.007, tp: 1.005, Close, 557.98, 36.7, 15.2, 71.65% 
 114 - imxusdt, sl: 1.007, tp: 1.005, High, 536.5, 36.71, 14.61, 67.72% 
 115 - imxusdt, sl: 1.007, tp: 0.999, Close, 1067.48, 230.75, 4.63, 62.42% 
 116 - imxusdt, sl: 1.007, tp: 0.999, High, 1036.03, 230.77, 4.49, 61.80% 
 117 - imxusdt, sl: 1.007, tp: 0.995, Close, 1640.34, 634.06, 2.59, 62.77% 
 118 - imxusdt, sl: 1.007, tp: 0.995, High, 1583.32, 634.09, 2.5, 62.52% 
 119 - imxusdt, sl: 1.007, tp: 0.99, Close, 1457.77, 1251.85, 1.16, 59.29% 
 120 - imxusdt, sl: 1.007, tp: 0.99, High, 1480.58, 1251.83, 1.18, 59.29% 
 121 - imxusdt, sl: 1.011, tp: 1.005, Close, 586.69, 36.69, 15.99, 75.20% 
 122 - imxusdt, sl: 1.011, tp: 1.005, High, 571.83, 36.7, 15.58, 72.80% 
 123 - imxusdt, sl: 1.011, tp: 0.999, Close, 1091.2, 230.74, 4.73, 64.88% 
 124 - imxusdt, sl: 1.011, tp: 0.999, High, 1044.31, 230.76, 4.53, 64.88% 
 125 - imxusdt, sl: 1.011, tp

 221 - ftmusdt, sl: 1.011, tp: 0.995, Close, 2033.07, 776.6, 2.62, 61.58% 
 222 - ftmusdt, sl: 1.011, tp: 0.995, High, 2067.75, 776.58, 2.66, 61.58% 
 223 - ftmusdt, sl: 1.011, tp: 0.99, Close, -388.81, 1447.87, -0.27, 58.46% 
 224 - ftmusdt, sl: 1.011, tp: 0.99, High, -221.3, 1447.79, -0.15, 58.46% 
 225 - adausdt, sl: 0.999, tp: 1.005, Close, 233.3, 19.07, 12.23, 60.56% 
 226 - adausdt, sl: 0.999, tp: 1.005, High, 246.51, 19.07, 12.93, 54.93% 
 227 - adausdt, sl: 0.999, tp: 0.999, Close, 1025.22, 299.94, 3.42, 60.81% 
 228 - adausdt, sl: 0.999, tp: 0.999, High, 1153.72, 299.87, 3.85, 48.41% 
 229 - adausdt, sl: 0.999, tp: 0.995, Close, 924.64, 1121.98, 0.82, 55.22% 
 230 - adausdt, sl: 0.999, tp: 0.995, High, 1922.38, 1121.48, 1.71, 51.41% 
 231 - adausdt, sl: 0.999, tp: 0.99, Close, -3566.43, 2071.95, -1.72, 47.62% 
 232 - adausdt, sl: 0.999, tp: 0.99, High, -1145.82, 2070.74, -0.55, 46.34% 
 233 - adausdt, sl: 1.003, tp: 1.005, Close, 234.15, 19.07, 12.28, 62.86% 
 234 - adausdt, s

In [20]:
results_df.to_csv(f'results_df_026.csv')